#Mount gogle drive, install dependancies and import libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

#Import dataset if using roboflow labeling service, otherwise load dataset into drive/content



In [ ]:
!curl -L "{DATASET URL}" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

#Register coco datasets

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/train/_annotations.coco.json", "/content/train"")
register_coco_instances("my_dataset_val", {}, "/content/val/_annotations.coco.json", "/content/val"")
register_coco_instances("my_dataset_test", {}, "/content/test/_annotations.coco.json", "/content/test")

#Create validation class

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
      """
      Method to build the COCO evaluator

      Parameters:
      cfg (CfgNode): Configuration for the training process
      dataset_name (str): Name of the dataset to evaluate on
      output_folder (str): Output folder to save results, default is None

      Returns:
      COCOEvaluator: Instance of COCOEvaluator
      """
    # If no output folder is provided, create one with name "coco_eval"
      if output_folder is None:
          os.makedirs("coco_eval", exist_ok=True)
          output_folder = "coco_eval"
      # Return an instance of COCOEvaluator 
      return COCOEvaluator(dataset_name, cfg, False, output_folder)
    


#Initialize model and create output directories 

In [ ]:
import os
from detectron2.config import get_cfg

# Configuring the model
cfg = get_cfg()

# Loading the configuration file for the model from the model zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))

# Setting the training and validation datasets
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

# Setting the number of workers for data loading
cfg.DATALOADER.NUM_WORKERS = 4

# Setting the weights for the model, initialized from the model zoo
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")

# Setting the solver parameters
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1500
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05

# Setting the parameters for ROI heads
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # Number of classes + 1

# Setting the evaluation period for testing
cfg.TEST.EVAL_PERIOD = 500

# Creating the output directory if it doesn't exist
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Initializing the trainer
trainer = CocoTrainer(cfg)

# Resuming from a checkpoint or loading from scratch
trainer.resume_or_load(resume=False)

#Train model and copy to google drive when finished. CHANGE YOUR DRIVE PATH

In [ ]:
# Run trainer
trainer.train()
!cp -r /content/output/model_final.pth /content/drive/MyDrive

#View training curves 

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

# Evaluate performance on test data

In [ ]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

# Setting the weights for the final model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

# Setting the score threshold for testing
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85

# Creating a predictor using the configuration
predictor = DefaultPredictor(cfg)

# Initializing the COCO evaluator
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")

# Building the test data loader
val_loader = build_detection_test_loader(cfg, "my_dataset_test")

# Evaluating the model on the test dataset
inference_on_dataset(trainer.model, val_loader, evaluator)